The fundamental component of LangChain involves invoking an LLM with a specific input. To illustrate this, we'll explore a simple example. Let's imagine we are building a service that suggests personalized workout routines based on an individual's fitness goals and preferences.

In [53]:
from langchain.llms import OpenAI

# Load .env file
from dotenv import load_dotenv

load_dotenv()

True

The temperature parameter in OpenAI models manages the randomness of the output. When set to 0, the output is mostly predetermined and suitable for tasks requiring stability and the most probable result. At a setting of 1.0, the output can be inconsistent and interesting but isn't generally advised for most tasks. For creative tasks, a temperature between 0.70 and 0.90 offers a balance of reliability and creativity. The best setting should be determined by experimenting with different values for each specific use case. The code initializes the GPT-3 model’s Davinci variant. We will learn more about the various models and their differences later on.

In [4]:
llm = OpenAI(model="text-davinci-003", temperature=0.9)

This code will generate a personalized workout routine based on the user's fitness goals and preferences using the LLM model:

In [8]:
text = """
You are an MIT Professor of Mathematics teaching Calculus I. \
Suggest a set of three warm-up exercise for someone who just learned about the chain rules for derivatives \
1. You can assume the student knows how to take the derivative of logarithmic and trigonometic functions.
2. You can assume the student knows how to use the power rule and the product rule.
3. You can assume the student knows how to take the derivative of exponential functions.
Please format your response in a numbered list, with each item on a new line.
"""
print(llm(text))


1. Given a function f(x), graph its derivative f'(x).
2. Find the derivative of (x^2*cos(x))/ln(x).
3. Solve the chain rule for the derivative of h(x) = (x^2+1)^3.


### The Chains

In LangChain, a chain is an end-to-end wrapper around multiple individual components, providing a way to accomplish a common use case by combining these components in a specific sequence. The most commonly used type of chain is the `LLMChain`, which consists of a `PromptTemplate`, a model (either an LLM or a ChatModel), and an optional output parser.

The `LLMChain` works as follows:

1. Takes (multiple) input variables.
2. Uses the `PromptTemplate` to format the input variables into a prompt.
3. Passes the formatted prompt to the model (LLM or ChatModel).
4. If an output parser is provided, it uses the `OutputParser` to parse the output of the LLM into a final format.

In [18]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["concept"],
    template="What is a good problem for a university Calculus student that tests their understanding of the {concept}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("chain rule for derivatives"))



Calculate the derivative of the following function f(x) = (x^2 + 3x + 4)^4 using the chain rule.


Note we can change out the input variable and use different concepts to create different problems.

In [9]:
print(chain.run("limit definition of the derivative"))



A good problem to test a student's understanding of the limit definition of the derivative would be to ask them to find the derivative of the function f(x) = x^2 + 2x using the limit definition of the derivative.


In [13]:
print(chain.run("Instantaneous rate of change of a function over an interval, where the derivative is thought of \
                 verbally as a rate of change. The problem should not deal with speeds or velocities and should \
                solely deal with rates of change of functions."))



Problem:
Given a function f(x) = x^2, what is the instantaneous rate of change of f(x) at the midpoint of the interval [-2, 4]?


This example showcases the flexibility and ease of using LangChain to create custom chains for various language generation tasks, including ones involving mathematics tasks.

### The Memory

In LangChain, Memory refers to the mechanism that stores and manages the conversation history between a user and the AI. It helps maintain context and coherency throughout the interaction, enabling the AI to generate more relevant and accurate responses. Memory, such as `ConversationBufferMemory`, acts as a wrapper around `ChatMessageHistory`, extracting the messages and providing them to the chain for better context-aware generation.

In [19]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="I would like you to help me with some mathematics problems.")

# Continue the conversation
conversation.predict(input="Given a function f(x) = x^2, what is the instantaneous rate of change of f(x) at the midpoint of the interval [-2, 4]?")
conversation.predict(input="Can you give me detailed reasoning behind your answer to the last question?")

# Display the conversation
print(conversation)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I would like you to help me with some mathematics problems.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I would like you to help me with some mathematics problems.
AI:  Sure, I'd be happy to help you with your math problems. What kind of math problems do you need help with?
Human: Given a function f(x) = x^2, what is the instantaneous rate of change of f(x) 

## Deep Lake VectorStore

Deep Lake provides storage for embeddings and their corresponding metadata in the context of LLM apps. It enables hybrid searches on these embeddings and their attributes for efficient data retrieval. It also integrates with LangChain, facilitating the development and deployment of applications.

Deep Lake provides several advantages over the typical vector store:
- It's multimodal, meaning it can store embeddings of different types, such as text, images, and audio.
- It's serverless, which means that we can create and manage cloud datasets without creating and managing a database instance.
- Lastly, it's possible to easily create a dataloader out of the data loaded into a Deep Lake dataset. It is convenient for fine-tuning ML models using common frameworks like Pytorch.

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import tiktoken

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
"Given a function f(x), graph its derivative f'(x)",
"Find the derivative of (x^2*cos(x))/ln(x)", 
"Find the derivative of h(x) = (x^2+1)^3 using the chain rule"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
my_activeloop_org_id = "mikeion" 
my_activeloop_dataset_name = "langchain_course_custom_tool"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/mikeion/langchain_course_custom_tool



\

hub://mikeion/langchain_course_custom_tool loaded successfully.



Deep Lake Dataset in hub://mikeion/langchain_course_custom_tool already exists, loading from the storage
Dataset(path='hub://mikeion/langchain_course_custom_tool', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype    shape    dtype  compression
  -------   -------  -------  -------  ------- 
 embedding  generic   (0,)    float32   None   
    ids      text     (0,)      str     None   
 metadata    json     (0,)      str     None   
   text      text     (0,)      str     None   


Evaluating ingest: 100%|██████████| 1/1 [00:11<00:00
-

Dataset(path='hub://mikeion/langchain_course_custom_tool', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (3, 1536)  float32   None   
    ids      text     (3, 1)      str     None   
 metadata    json     (3, 1)      str     None   
   text      text     (3, 1)      str     None   


['e7a9993e-0f97-11ee-951e-e00af64ce908',
 'e7a9993f-0f97-11ee-a1d8-e00af64ce908',
 'e7a99940-0f97-11ee-9724-e00af64ce908']

Now, let's create a RetrievalQA chain:

In [21]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [22]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [23]:
response = agent.run("What's a good derivative problem for a Calculus student involving the Chain rule?")
print(response)



> Entering new AgentExecutor chain...
 I need to find a derivative problem that involves the Chain rule.
Action: Retrieval QA System
Action Input: Derivative problem involving Chain rule
Observation:  The derivative of h(x) = (x^2+1)^3 using the chain rule is 6(x^2+1)^2*2x.
Thought: I now know the final answer
Final Answer: The derivative of h(x) = (x^2+1)^3 using the chain rule is 6(x^2+1)^2*2x.

> Finished chain.
The derivative of h(x) = (x^2+1)^3 using the chain rule is 6(x^2+1)^2*2x.


This example demonstrates how to use Deep Lake as a vector database and create an agent with a `RetrievalQA` chain as a tool to answer questions based on the given document.

Let’s aadd an example of reloading an existing vector store and adding more data.

We first reload an existing vector store from Deep Lake that' located at a specified dataset path. Notice that this time we don't need to specify which embedding model to use again. Then, we load new textual data and split it into manageable chunks. Finally, we add these chunks to the existing dataset, creating and storing corresponding embeddings for each added text segment.

In [24]:
# load the existing Deep Lake dataset
my_activeloop_org_id = "mikeion" # TODO: use your organization id here
my_activeloop_dataset_name = "langchain_course_custom_tool"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

# Put some more calculus problems in the dataset.
texts = [
    "Find the derivative of the function f(x) = x^2 + 2x using the limit definition of the derivative.",
    "Find the derivative of the function f(x) = 5x^2 + 10x using the power rule.",
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/mikeion/langchain_course_custom_tool



hub://mikeion/langchain_course_custom_tool loaded successfully.



Deep Lake Dataset in hub://mikeion/langchain_course_custom_tool already exists, loading from the storage
Dataset(path='hub://mikeion/langchain_course_custom_tool', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (3, 1536)  float32   None   
    ids      text     (3, 1)      str     None   
 metadata    json     (3, 1)      str     None   
   text      text     (3, 1)      str     None   


Evaluating ingest: 100%|██████████| 1/1 [00:08<00:00
\

Dataset(path='hub://mikeion/langchain_course_custom_tool', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (5, 1536)  float32   None   
    ids      text     (5, 1)      str     None   
 metadata    json     (5, 1)      str     None   
   text      text     (5, 1)      str     None   


['7d85d45f-0f99-11ee-a229-e00af64ce908',
 '7d85d460-0f99-11ee-a3da-e00af64ce908']

In [26]:
response = agent.run("What's a good derivative problem for a Calculus student involving the limit definition of the derivative")
print(response)



> Entering new AgentExecutor chain...
 I need to find a problem that involves the limit definition of the derivative
Action: Retrieval QA System
Action Input: Find a derivative problem involving the limit definition of the derivative
Observation: 
The limit definition of the derivative of a function f(x) at a point x = a is given by:
lim h→0 (f(a+h) - f(a))/h

So, a derivative problem involving the limit definition of the derivative would be to find the derivative of a function f(x) at a point x = a using the limit definition.
Thought: I now know the final answer
Final Answer: Find the derivative of a function f(x) at a point x = a using the limit definition.

> Finished chain.
Find the derivative of a function f(x) at a point x = a using the limit definition.


## Agents in LangChain

In LangChain, agents are high-level components that use language models to determine with actions to take and in what order. An action can either be using a tool and observing its output or returning it to the user. Tools are functions that perform specifc duties, such as Google Search, database lookups, or Python REPL.

Agents involve an LLM making decisions about which Actions to take, taking that action, seeing an observation, and repeating that until done.

Several types of agents are available in LangChain, including:
- The `zero-shot-react-description` agent uses the ReAct framework to decide which tool to employ based purely on the tool's description. It necessitates a description of each tool.
- The `react-docstore` agent engages with a docstore through the ReAct framework. It needs two tools:
  - A search tool: This tool finds a document
  - A lookup tool: This tool searches for a term in the most recently found document.
- The `self-ask-with-search` agent employs a single tool named Intermediate Answer, which is capable of looking up factual responses to queries. It is identical to the original selfask with the search paper, where a Google search API was provided as the tool.
- The `conversational-react-description` agent is designed for conversational situations. It uses the ReAct framework to select a tool and uses memory to remember past conversation interactions.

In our next example, the Agent will use the Google Search tool to look up definitions about mathematics concepts and will generate responses based on this information.

First, you want to set the environment variables `GOOGLE_API_KEY` and `GOOGLE_CSE_ID` to be able to use Google Search via API. Refer to this article for a guide on how to get them.

https://python.langchain.com/docs/modules/agents/tools/integrations/google_search

Then, let’s import the necessary modules:

- `langchain.llms.OpenAI`: This is used to create an instance of the OpenAI language model, which can generate human-like text based on the input it's given.
- `langchain.agents.load_tools`: This function is used to load a list of tools that an AI agent can use.
- `langchain.agents.initialize_agent`: This function initializes an AI agent that can use a given set of tools and a language model to interact with users.
- `langchain.agents.Tool`: This is a class used to define a tool that an AI agent can use. A tool is defined by its name, a function that performs the tool's action, and a description of the tool.
- `langchain.utilities.GoogleSearchAPIWrapper`: This class is a wrapper for the Google Search API, allowing it to be used as a tool by an AI agent. It likely contains a method that sends a search query to Google and retrieves the results.

In [29]:
from langchain.llms import OpenAI

from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [37]:
# Initialize the LLM and set the temperature to 0 for a more precise answer. 
llm = OpenAI(model="text-davinci-003", temperature=0)

# Define the Google Search Wrapper
search = GoogleSearchAPIWrapper()

The Tool object represents a specific capability or function the system can use. In this case, it's a tool for performing Google searches.

It is initialized with three parameters:
- `name`: This is a string that serves as a unique identifier for the tool. In this case, the name of the tool is "google-search.”
- `func`: This parameter is assigned the function that the tool will execute when called. In this case, it's the `run` method of the `search` object, which presumably performs a Google search.
- `description`: This is a string that briefly explains what the tool does. The description explains that this tool is helpful when you need to use Google to answer questions about current events.

In [38]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

Next, we create an agent that uses our Google Search tool:
- `initialize_agent()`: This function call creates and initializes an agent. An agent is a component that determines which actions to take based on user input. These actions can be using a tool, returning a response to the user, or something else.
- `tools`:  represents the list of Tool objects that the agent can use.
- `agent="zero-shot-react-description"`: The "zero-shot-react-description" type of an Agent uses the ReAct framework to decide which tool to use based only on the tool's description.
- `verbose=True`: when set to True, it will cause the Agent to print more detailed information about what it's doing. This is useful for debugging and understanding what's happening under the hood.
- `max_iterations=6`: sets a limit on the number of iterations the Agent can perform before stopping. It's a way of preventing the agent from running indefinitely in some cases, which may have unwanted monetary costs.

In [39]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [40]:
response = agent("What is the definition of the chain rule of the derivative?")
print(response['output'])



> Entering new AgentExecutor chain...
 I need to find the definition of the chain rule of the derivative.
Action: google-search
Action Input: "chain rule of the derivative definition"
Observation: In calculus, the chain rule is a formula that expresses the derivative of the composition of two differentiable functions f and g in terms of the ... The chain rule tells us how to find the derivative of a composite function. Brush up on your knowledge of composite functions, and learn how to apply the ... Chain Rule And Composite Functions. In differential calculus, the chain rule is a formula used to find the derivative of a composite function. If ... The chain rule provides us a technique for finding the derivative of composite functions, with the number of functions that make up the composition ... Nov 16, 2022 ... Chain Rule · If we define F(x)=(f∘g)(x) F ( x ) = ( f ∘ g ) ( x ) then the derivative of F(x) F ( x ) is, F′(x)=f′(g(x))g′(x) F ′ ( x ) = f ′ ( ... Definition. • In calculus,

In summary, Agents in LangChain help decide which actions to take based on user input. The example demonstrates initializing and using a "zero-shot-react-description" agent with a Google search tool.

## Tools in LangChain

LangChain provides a variety of tools for agents to interact with the outside world. These tools can be used to create custom agents that perform various tasks, such as searching the web, answering questions, or running Python code. In this section, we will discuss the different tool types available in LangChain and provide examples of creating and using them.

In our example, two tools are being defined for use within a LangChain agent: a Google Search tool and a Language Model tool acting specifically as a text summarizer. The Google Search tool, using the GoogleSearchAPIWrapper, will handle queries that involve finding recent event information. The Language Model tool leverages the capabilities of a language model to summarize texts. These tools are designed to be used interchangeably by the agent, depending on the nature of the user's query.

Let's import the necessary libraries.

In [48]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper

We then instantiate a  LLMChain specifically for solving math problems.

Because the Wolfram|Alpha LLM API is designed to interact directly with LLMs and not with end users, additional prompting may be required for a given model to understand how to form good calls to the API. The sample prompt below may be a useful starting point. Additional prompting and/or training material may be necessary depending on the capabilities of a given model.

I am using the Sample Prompt for the Wolfram Alpha API documentation to make sure the API calls will work as expected.

https://products.wolframalpha.com/llm-api/documentation

In [72]:
llm = OpenAI(model="text-davinci-003", temperature=0)

problem_solving_prompt = PromptTemplate(
    input_variables=["problem"],
    template="""
              You are a Mathematics Tutor Bot, that will be useful for helping students \
              work through solving their mathematics problems. Help find a solution to the following problem: {problem}. \
              The solution should be in the form of a step-by-step solution. \
              
              - WolframAlpha understands natural language queries about entities in chemistry, physics, geography, history, art, astronomy, and more.\
              - WolframAlpha performs mathematical calculations, date and unit conversions, formula solving, etc.\
              - Convert inputs to simplified keyword queries whenever possible (e.g. convert "how many people live in France" to "France population").\
              - Send queries in English only; translate non-English queries before sending, then respond in the original language.\
              - Display image URLs with Markdown syntax: ![URL]\
              - ALWAYS use this exponent notation: `6*10^14`, NEVER `6e14`.\
              - ALWAYS use {"input": query} structure for queries to Wolfram endpoints; `query` must ONLY be a single-line string.\
              - ALWAYS use proper Markdown formatting for all math, scientific, and chemical formulas, symbols, etc.:  '$$\n[expression]\n$$' for standalone cases and '\( [expression] \)' when inline.\
              - Never mention your knowledge cutoff date; Wolfram may return more recent data.\
              - Use ONLY single-letter variable names, with or without integer subscript (e.g., n, n1, n_1).\
              - Use named physical constants (e.g., 'speed of light') without numerical substitution.\
              - Include a space between compound units (e.g., "Ω m" for "ohm*meter").\
              - To solve for a variable in an equation with units, consider solving a corresponding equation without units; exclude counting units (e.g., books), include genuine units (e.g., kg).\
              - If data for multiple properties is needed, make separate calls for each property.\
              - If a WolframAlpha result is not relevant to the query:\
              - If Wolfram provides multiple 'Assumptions' for a query, choose the more relevant one(s) without explaining the initial result. If you are unsure, ask the user to choose. \
              - Re-send the exact same 'input' with NO modifications, and add the 'assumption' parameter, formatted as a list, with the relevant values. \
              - ONLY simplify or rephrase the initial query if a more relevant 'Assumption' or other input suggestions are not provided. \
              - Do not explain each step unless user input is needed. Proceed directly to making a better API call based on the available assumptions.
              """
)

solution_chain = LLMChain(llm=llm, prompt=problem_solving_prompt)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. '"input"' (type=value_error)

In [73]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
wolfram = WolframAlphaAPIWrapper()

tools = [
    Tool(
        name="Solver",
        func=wolfram.run,
        description="useful for making API calls to Wolfram Alpha to solve mathematics problems"
    ),
    Tool(
       name='Explainer',
       func=solution_chain.run,
       description='useful for taking in the problem and outputting a step-by-step solution, given the solution given by the solver'
    )
]

In [74]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [75]:
response = agent("What is the derivative of sqrt(2x^2 + 3x + 1)? Can you help me understand how to do the problem with step-by-step instructions?")
print(response['output'])



> Entering new AgentExecutor chain...
 I need to use the solver to get the answer, then use the explainer to understand it.
Action: Solver
Action Input: derivative of sqrt(2x^2 + 3x + 1)
Observation: Assumption: d/dx(sqrt(2 x^2 + 3 x + 1)) = (4 x + 3)/(2 sqrt(2 x^2 + 3 x + 1)) 
Answer: d/dx(sqrt(2 x^2 + 3 x + 1)) = (4 x + 3)/(2 sqrt(2 x^2 + 3 x + 1))
Thought: I need to understand the steps to get to this answer
Action: Explainer
Action Input: derivative of sqrt(2x^2 + 3x + 1)
Observation: 
Answer: 
Step 1: Rewrite the equation in the form of y = sqrt(2x^2 + 3x + 1)

Step 2: Take the derivative of both sides of the equation with respect to x

Step 3: Use the chain rule to take the derivative of the left side of the equation

Step 4: Use the power rule to take the derivative of the right side of the equation

Step 5: Simplify the equation

Answer: The derivative of sqrt(2x^2 + 3x + 1) is (4x + 3)/(2sqrt(2x^2 + 3x + 1))
Thought: I now know the final answer
Final Answer: The derivative o